In [11]:
import requests
import json
from bs4 import BeautifulSoup
from csv import writer

client_id = 'client_80e72e5511155fdde9a18e467f829c15'
client_secret = 'secret_4e3af2f4c36e901b8f7c0afbda475f32'
scopes = ['api_listings_read']
auth_url = 'https://auth.domain.com.au/v1/connect/token'
url_endpoint = 'https://api.domain.com.au/v1/listings/residential/_search'

response = requests.post(auth_url, data = {
                        'client_id':client_id,
                        'client_secret':client_secret,
                        'grant_type':'client_credentials',
                        'scope':scopes,
                        'Content-Type':'text/json'
                        })
json_res = response.json()
access_token=json_res['access_token']
print(access_token)
auth = {'Authorization':'Bearer ' + access_token}
url = url_endpoint

#areas = ["Melbourne City Council - Greater Area",
#                  "Port Philip City Council - Greater Area",
#                  "Stonnington City Council - Greater Area",
#                  "Yarra City Council - Greater Area"]
post_fields = {
      "listingType":"Rent",
      "pageSize": 200,
      #"pageNumber": 2,
      "propertyTypes":"",
      "minBedrooms":1,
      "minBathrooms":1,
      #"maxPrice": 400,
      "sort": {
          "sortKey": "Default"
      },
      
      "locations":[
        {
          "state":"VIC",
          "region":"",
          #"area":"Melbourne City Council - Greater Area",
          #"area":"",
          "suburb":"",
          "postCode":"",
          "includeSurroundingSuburbs":False
        }
      ]
}

# Request content
res1 = requests.post(url,headers=auth,json=post_fields)
print(res1)
content = json.loads(res1.text)
print(len(content))


d6f361c8f83fc5158fafcaf5dcbe5379
<Response [200]>
200


In [10]:
#content

{'errors': {'searchParameters.PageSize': ["'Page Size' must be less than or equal to '200'."]},
 'message': 'The request is invalid.'}

In [12]:
import csv

# Write column headers
headers = ['street_adress', 'suburb', 'postcode', 'area', 'property_type', 'bedrooms', 'bathrooms', 'carspaces', 'price', 'time', 'latitude', 'longitude']
output_csv = open('lst1.csv', 'w')
# create the csv writer object
csv_writer = csv.writer(output_csv)
# first write the headers
csv_writer.writerow(headers)

output_csv.close()

# Get request for each property ID and extract attributes
for i in content:
    #get details
    r = i['listing']['propertyDetails']
    street_address=r['displayableAddress']
    suburb=r['suburb']
    postcode=r['postcode']
    area=r['area']
    latitude = r['latitude']
    longitude = r['longitude']
    property_type=r['propertyType']
    bathrooms=r['bathrooms']
    bedrooms=r['bedrooms']
    
    #print(i['listing']['inspectionSchedule']['times'])
    if(i['listing']['inspectionSchedule']['times']):
        time = i['listing']['inspectionSchedule']['times'][0]['openingTime']
    else:
        time = 0

    if(str(i).find('carspaces'))>0:
        carspaces=i['listing']['propertyDetails']['carspaces']
    else:
        carspaces=0
    
    price=i['listing']['priceDetails']['displayPrice'][1:4]

    info = [street_address, suburb, postcode, area, property_type, bedrooms, bathrooms, carspaces, price, time, latitude, longitude]
    
    # append the remaining rows one by one
    with open('lst1.csv', 'a', newline='') as g:
        thewriter = writer(g)        
        thewriter.writerow(info)


